In [23]:
import os
import win32com.client
import numpy as np
import re

def replace_E(s, new_number):
    # Find the position of the last operator in the string
    last_operator_match = re.search(r'[\+\-](?=[^+\-]*$)', s)
    last_operator_pos = last_operator_match.start()
    # Find the end position of the number following the last operator
    next_operator_match = re.search(r'[\+\-]', s[last_operator_pos+1:])
    if next_operator_match:
        end_pos = last_operator_pos + next_operator_match.start() + 1
    else:
        end_pos = len(s)
    # Replace the last parameter including its sign
    new_string = s[:last_operator_pos] + str(new_number) + s[end_pos:]
    return new_string

#define groups through which to access all utility functions
VHG_EinP = ['EinpA_mF','EinpA_oF','EinpF_mF','EinpF_oF','EinpK_mF','EinpK_oF','EinpU_mF','EinpU_oF']
Mode = ['bic']

VHG_main = [
"BSch_mF", "BSch_oF", "ERWmP_mF", "ERWmP_oF", "ERWoP_mF", "ERWoP_oF", "GSch_mF", "GSch_oF", "Kind_mF", "Kind_oF",
"NERWmP_mF", "NERWmP_oF", "NERWoP_mF", "NERWoP_oF", "Rentner_mF", "Rentner_oF", "Sch_mF", "Sch_ko", "Stud_mF", "Stud_oF"
]
Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]

#Parameter values for E to try out (see Excel 003)
E = [['+0.5'],
    ['+0.6'],
    ['+0.2'],
    ['+0.5'],
    ['-0.3']]

# Matrix numbers for calculating resulting modal share per trip purpose
mAW_bic = 5292
mAx_bic = 5221
mF_bic = 5291
mFx_bic = 5222
mBEK_bic = 5293
mKx_bic = 5223
mGJSU_bic = 5294
mUx_bic = 5224
mHY_bic = 5295

mAW_all = 122
mAx_all = 20
mF_all = 126
mFx_all = 21
mBEK_all = 125
mKx_all = 22
mGJSU_all = 128
mUx_all = 23
mHY_all = 132

In [ ]:
#Loop over iteration steps to change E, rerun model, compute mode shares by trip purpose, and save
for i in range(0,1):
    #open ver
    #verPath = r'D:\ver\20240709_1200.ver'
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250116_0845_E3_0.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    
    #change utils for Einpendler visem
    visem_einp = Visum.Procedures.Operations.ItemByKey(44)
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":

                # Set E based on trip purpose
                if vhg == 'EinpA_mF' or vhg == 'EinpA_oF':
                    if mode == 'bic':
                        new_E = E[0][i]
                elif vhg == 'EinpK_mF' or vhg == 'EinpK_oF':
                    if mode == 'bic':
                        new_E = E[2][i]
                elif vhg == 'EinpF_mF' or vhg == 'EinpF_oF':
                    if mode == 'bic':
                        new_E = E[1][i]
                elif vhg == 'EinpU_mF' or vhg == 'EinpU_oF':
                    if mode == 'bic':
                        new_E = E[3][i]
                
                #write new util function into Visum
                new_util = replace_E(util,new_E)
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)
                
    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(45)
    
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exception
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    if util != "-99999":
                        if mode == 'bic':
                            if activity == 'A' or activity == 'Ax' or activity == 'W':
                                new_E = E[0][i]
                            elif activity == 'F' or activity == 'Fx':
                                new_E = E[1][i]
                            elif activity == 'B' or activity == 'E' or activity == 'K' or activity == 'Kx':
                                new_E = E[2][i]
                            elif activity == 'G' or activity == 'J' or activity == 'S' or activity == 'U' or activity == 'Ux':
                                new_E = E[3][i]
                            elif activity == 'H' or activity == 'Y':
                                new_E = E[4][i]                      

                        #write new utilityfunction into visum
                        new_util = replace_E(util,new_E)
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)

                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    #run procedures
    #Visum.Procedures.Execute()
    
    # Calculate mode share per persongroup
    print('iteration '+str(i)+'; A, Ax, W:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mAW_bic).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_bic).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mAW_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mAx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; F, Fx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mF_bic).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_bic).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mF_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mFx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; B, E, K, Kx:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mBEK_bic).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_bic).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mBEK_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mKx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; G, J, S, U, Ux:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mGJSU_bic).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_bic).GetValues()).sum())/
                 (np.array(Visum.Net.Matrices.ItemByKey(mGJSU_all).GetValues()).sum() + np.array(Visum.Net.Matrices.ItemByKey(mUx_all).GetValues()).sum()))*100, 1)
         )
    print('iteration '+str(i)+'; H, Y:')
    print(round(((np.array(Visum.Net.Matrices.ItemByKey(mHY_bic).GetValues()).sum()) /
                 (np.array(Visum.Net.Matrices.ItemByKey(mHY_all).GetValues()).sum())) * 100, 1)
         )

    #save
    #verPath_append = r'D:\ver\20240709_1200_'+str(i)+'.ver'
    verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250116_0845_E3_'+str(i)+'.ver'
    Visum.saveVersion(verPath_append)
    Visum = None